In [ ]:
import MetaTrader5 as mt5 #mt5 to access historcal data
import pandas as pd # for data analysis and calculation of technical indcator
import numpy as np
import plotly.io as pio
import plotly.express as px # for data visualization
import plotly.graph_objects as go
import tdclient as TDClient
from datetime import datetime, time ,timezone #to specify the date time range for historical data
from IPython.display import display, Markdown, Latex # to display result in python notebook
from backtest import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

: 

In [ ]:
# Conecte mt5
mt5.initialize()
# logine mt5
login = 52185665
password = '3cAJz$AQiKwMq0'
server = 'ICMarketsSC-Demo'

mt5.login(login,password,server)

In [ ]:
# this function retreives olhc data from mt5 account and return a data frame
def get_ohlc(symbol, timeframe, start_datetime, end_datetime):
    ohlc = mt5.copy_rates_range(symbol, timeframe, start_datetime, end_datetime)
    ohlc_df = pd.DataFrame(ohlc)
    ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit= 's')
    return ohlc_df[['time', 'open', 'high', 'low', 'close']]


In [ ]:
# 1 minut df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_M1
start_datetime = datetime(2025,3,13)
end_datetime = datetime.now()
ohlc_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
ohlc_df

In [ ]:
# hour4 df
timeframe = mt5.TIMEFRAME_H4
hourly_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
hourly_df.tail(5)

In [ ]:
# daily df
timeframe = mt5.TIMEFRAME_D1
daily_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
daily_df

In [ ]:
# Calculate Pivot Point, R1, R2, R3, stopeR4, S1, S2, S3, stopeS4 levels with Fibonacci 
daily_df["pivot"] = ((daily_df["high"] + daily_df["low"] + daily_df["close"]) / 3).shift(1)
daily_df["prange"] = (daily_df["high"] - daily_df["low"]).shift(1)
daily_df["R1"] = daily_df["pivot"] + 0.382 * daily_df["prange"]
daily_df["R2"] = daily_df["pivot"] + 0.618 * daily_df["prange"]
daily_df["R3"] = daily_df["pivot"] + 1.000 * daily_df["prange"]

daily_df["S1"] = daily_df["pivot"] - 0.382 * daily_df["prange"]
daily_df["S2"] = daily_df["pivot"] - 0.618 * daily_df["prange"]
daily_df["S3"] = daily_df["pivot"] - 1.000 * daily_df["prange"]
daily_df[["time","S1","R1"]].tail(10)


In [ ]:
#setting the sma 1 and 14 periods
furst_sma_period = 1
second_sma_period = 14
#ohlc4
hourly_df['ohlc4'] = (hourly_df['open'] + hourly_df['high'] + hourly_df['low'] + hourly_df['close']) / 4

# sma 1
hourly_df['sma_1'] = hourly_df['ohlc4'].rolling(furst_sma_period).mean()
hourly_df['prev_sma_1'] = hourly_df['sma_1'].shift(1) #to find crossovers, previous sma value is necessary using shift()
# sma 14
hourly_df['sma_14'] = hourly_df['ohlc4'].rolling(second_sma_period).mean()
hourly_df

In [ ]:
#setting the 11 and 114 sma periods
thurd_sma_period = 1
fourth_sma_period = 14
#ohlc4
daily_df['ohlc4'] = (daily_df['open'] + daily_df['high'] + daily_df['low'] + daily_df['close']) / 4
# sma 11
daily_df['sma_11'] = daily_df['ohlc4'].rolling(thurd_sma_period).mean()
daily_df['prev_sma_11'] = daily_df['sma_11'].shift(1) #to find crossovers, previous sma value is necessary using shift()
# sma 114
daily_df['sma_114'] = daily_df['ohlc4'].rolling(fourth_sma_period).mean()
daily_df[["time","sma_11","sma_114","prev_sma_11"]]

In [ ]:
#setting entring and closing time and one trade a day
# Set the start and end times for your desired trading hours
start_time = datetime.strptime('06:00:00', '%H:%M:%S').time()
end_time = datetime.strptime('17:00:00', '%H:%M:%S').time()
end_trading_section = datetime.strptime('23:45:00', '%H:%M:%S').time()

# Filter the trade data for trades within the specified trading hours
#trades_within_hours = daily_df[(daily_df['time'].dt.time >= start_time) & (daily_df['time'].dt.time <= end_time)]
# open one trade a day
first_trade_per_day = daily_df.groupby(daily_df['time'].dt.time).first()
# Close all open positions at 11:45 PM
open_positions = daily_df[daily_df['time'].dt.time == end_trading_section]

# function to trading
daily_df["sell_stope_loss"] = (daily_df["R2"] - daily_df["S1"]) /2 + daily_df["R2"] 
daily_df["buy_stope_loss"] = (daily_df["S2"] - ((daily_df["R1"] - daily_df["S2"]) /2))
daily_df['sell_profit'] = daily_df['low'] <= daily_df["S1"]
daily_df['buy_profit'] = daily_df["high"] >= daily_df["R1"]
daily_df

In [ ]:
def entring_stoploss_profit(row, ohlc_df) :
    if row["prev_sma_11"] > row['sma_114'] and row["low_x"] <= row["S1"] :
        return "buy"
    elif row['prev_sma_11'] < row['sma_114'] and row["high_x"] >= row["R1"]:
        return "sell"
    # Merge daily indicators into minute data (asof = merge on closest earlier date)
merged_df = pd.merge_asof(ohlc_df, daily_df, on='time', direction='backward')
merged_df["signal"] = merged_df.apply(entring_stoploss_profit, axis=1, ohlc_df = ohlc_df)

# Tag first signal per day in original merged_df
merged_df['is_first_signal'] = (merged_df[merged_df["signal"].notna()].groupby(merged_df["time"].dt.date).cumcount() == 0)

# Set signal only on the first row per day, others = None
merged_df['signal'] = merged_df.apply(lambda row: row['signal'] if row['is_first_signal'] else None, axis=1)
merged_df[['signal', 'time']].dropna()

In [ ]:
merged_df = merged_df.rename(columns={
    "low_x": "low",
    "high_x": "high",
    "open_x": "open",
    "close_x": "close",
    "low_y": "daily_low",
    "high_y": "daily_high",
    "open_y": "daily_open",
    "close_y": "daily_close"
})


In [ ]:
print(merged_df.columns.tolist())
merged_df


In [ ]:
# create trade logic
def on_bar(data, trades, orders):
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    if 'signal' not in data:
        print("Warning: 'signal' column is missing!")
        return
    account_balance = 10000  # your account in USD
    risk_percent = 0.20  # 2%
    pip_value_per_lot = 10  # on EURUSD, 1 lot = $10 per pip
    risk_amount = account_balance * risk_percent
    # entry signal
    if data['signal'] == 'buy' and not num_open_trades:
        stop_loss_pips = ((data["R1"] - data["S1"]) /2) # example SL
        volume = risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'buy')
    
    elif data['signal'] == 'sell' and not num_open_trades:
        stop_loss_pips = ((data["R1"] - data["S1"]) /2)
        volume = risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'sell')
        
# exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and ((data["low"] <= data["S1"] - ((data["R1"] - data["S1"]) /2))
                                            or (data["high"] >= data["R1"])
                                            or (data['time'].time() == pd.to_datetime('00:00:00').time())):
            orders.close_trade(trade)
        elif trade['order_type'] == 'sell' and ((data["high"] >= (data["R1"] - data["S1"]) /2 + data["R1"])
                                            or (data["low"] <= data["S1"])
                                            or (data['time'].time() == pd.to_datetime('00:00:00').time())):
            orders.close_trade(trade)

In [ ]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -7 / 100000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(merged_df)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades

In [ ]:
mm = bt.plot_balance()
display(mm)

In [ ]:
pnl_chart = bt.plot_pnl()
display(pnl_chart)

In [ ]:
evaluate_backtest(bt.trades)

In [ ]:
# visualizing the ohlc data
fig = go.Figure(data = [go.Candlestick(x = ohlc_df['time'],
                                        open = ohlc_df['open'],
                                        high = ohlc_df['high'],
                                        low = ohlc_df['low'],
                                        close = ohlc_df['close'])])

# fig.update_layout(height=600, xaxis_rangeslider_visible=False)
# 📌 Add Pivot Points as Horizontal Lines
pivot_levels = ["sell_stope_loss", "R2", "buy_stope_loss", "S2"]
colors = {'sell_stope_loss': 'red', 'buy_stope_loss': 'red', 'R2': 'green', 'S2': 'green'}


for _, row in daily_df.iterrows():
    for level in pivot_levels:
        fig.add_trace(go.Scatter(
            x=[row['time'], row['time'] + pd.Timedelta(days=1)],  # Extend for one day
            y=[row[level], row[level]],
            mode='lines',
            name=f"{level} {row['time']}",
            line=dict(color=colors[level], width=1.5, dash='dot')
        ))

# 📌 Customize Layout
fig.update_layout(
    title="Pivot Points on Candlestick Chart",
    xaxis_title="Date",
    yaxis_title="Price",
    template="plotly_dark",
    xaxis_rangeslider_visible=False
)
fig